# Sprint 機械学習スクラッチ入門

**Sprintの目的**

- 機械学習スクラッチの準備をする

**どのように学ぶか**

- 今後の機械学習スクラッチ課題で作成するモデルを、scikit-learnを用いて一度動かしておきます。これまでの復習を兼ねたスクラッチ課題の準備です。

## 【問題1】train_test_splitのスクラッチ

スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。


[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
def scratch_train_test_split(X,y,train_size=0.8,random_state=0):
    """
    検証データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    # Xとyのデータサイズが一致しないとエラーメッセージ
    assert X.shape[0] == y.shape[0], "Xとyのデータサイズが一致しない"
    
    # 乱数のシード
    np.random.seed(random_state)
    
    # X,yを一旦横方向に結合させる
    y = y.reshape(-1,1)
    Xy = np.concatenate([X,y],axis=1)
    
    # trainデータ、testデータの分割
    Xy_size = len(Xy)
    pick_num = int(np.round(Xy_size*train_size))
    train_pick = np.random.choice(a=np.arange(Xy_size),size=pick_num,replace=False)
    test_pick = np.delete(np.arange(Xy_size),train_pick)
    
    train = Xy[train_pick,:]
    test = Xy[test_pick,:]

    X_train = train[:,0:(Xy.shape[1]-y.shape[1])].reshape(-1,X.shape[1])
    y_train = train[:,-y.shape[1]].reshape(-1,)
    X_test = test[:,0:(Xy.shape[1]-y.shape[1])].reshape(-1,X.shape[1])
    y_test = test[:,-y.shape[1]].reshape(-1,)
    
    return X_train, X_test, y_train, y_test  

sklearn.model_selection.train_test_split公式の例(np.ndarray) 

In [3]:
X, y = np.arange(10).reshape((5, 2)), range(5)
print(X)
print(list(y))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[0, 1, 2, 3, 4]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train)
print(y_train)

[[4 5]
 [0 1]
 [6 7]]
[2, 0, 3]


In [5]:
print(X_test)
print(y_test)

[[2 3]
 [8 9]]
[1, 4]


scratch_train_test_split関数で検証

In [6]:
X, y = np.arange(10).reshape((5, 2)), range(5)
print(X)
print(list(y))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[0, 1, 2, 3, 4]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train)
print(y_train)

[[4 5]
 [0 1]
 [6 7]]
[2, 0, 3]


In [8]:
print(X_test)
print(y_test)

[[2 3]
 [8 9]]
[1, 4]


## 【問題2】 分類問題を解くコードの作成

In [9]:
#irisデータセットを読み込む
from sklearn.datasets import load_iris

iris_dataset = load_iris()
X = pd.DataFrame(iris_dataset.data, columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
y = pd.DataFrame(iris_dataset.target, columns=["Species"])
df = pd.concat([X, y], axis=1)

df_selected = df.query("Species == [1, 2]").loc[:, ["sepal_length", "petal_length", "Species"]]
df_selected.head()

,sepal_length,petal_length,Species
50,7.0,4.7,1
51,6.4,4.5,1
52,6.9,4.9,1
53,5.5,4.0,1
54,6.5,4.6,1


In [10]:
# スクラッチしたtrain_test_split関数でtrainデータとtestデータに分ける

X_train, X_test, y_train, y_test = scratch_train_test_split(df_selected.iloc[:, :-1].values, df_selected.iloc[:, -1:].values, train_size=0.75)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

X_train (75, 2)
X_test (25, 2)
y_train (75,)
y_test (25,)


In [11]:
# 前処理・標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # インスタンス作成
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train) #X_trainの標準化
X_test_transformed = scaler.transform(X_test) #X_testの標準化
# print(X_train_transformed)
# print(X_test_transformed)

In [12]:
# irisデータセットの学習・推定：ロジスティック回帰(勾配降下法SGDClassifierクラス使用)
from sklearn.linear_model import SGDClassifier

sgdc = SGDClassifier(loss="log")
sgdc.fit(X_train_transformed, y_train)
sgdc_pred = sgdc.predict(X_test_transformed)
sgdc_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2.,
       2., 1., 2., 2., 2., 1., 2., 2.])

In [13]:
# irisデータセットの学習・推定：SVM
from sklearn.svm import SVC

svm = SVC() # インスタンス作成
svm.fit(X_train_transformed, y_train) # 学習
svm_pred = svm.predict(X_test_transformed) # 予測
svm_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 1., 2., 2.])

In [14]:
# irisデータセットの学習・推定：決定木
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier() # インスタンス作成
dtc.fit(X_train_transformed, y_train) # 学習
dtc_pred = dtc.predict(X_test_transformed) # 予測
dtc_pred

array([1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 1.])

### シンプルデータセット1での学習・推定

In [15]:
# サンプルコードでシンプルデータセット1作成

import numpy as np
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]
display(X.shape)
display(y.shape)

(500, 2)

(500,)

In [16]:
# スクラッチしたtrain_test_split関数でtrainデータとtestデータに分ける

X_train, X_test, y_train, y_test = scratch_train_test_split(X, y, train_size=0.75)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

X_train (375, 2)
X_test (125, 2)
y_train (375,)
y_test (125,)


In [17]:
# 前処理・標準化

scaler = StandardScaler() # インスタンス作成
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train) #X_trainの標準化
X_test_transformed = scaler.transform(X_test) #X_testの標準化
# print(X_train_transformed)
# print(X_test_transformed)

In [18]:
# シンプルデータセット1の学習・推定：ロジスティック回帰(勾配降下法SGDClassifierクラス使用)

sgdc = SGDClassifier(loss="log")
sgdc.fit(X_train_transformed, y_train)
sgdc_pred = sgdc.predict(X_test_transformed)
sgdc_pred

array([-1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,
       -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
       -1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
       -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1.,  1.])

In [19]:
# シンプルデータセット1の学習・推定：SVM

svm = SVC() # インスタンス作成
svm.fit(X_train_transformed, y_train) # 学習
svm_pred = svm.predict(X_test_transformed) # 予測
svm_pred

array([-1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,
       -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
       -1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
       -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1.,  1.])

In [20]:
# シンプルデータセット1の学習・推定：決定木

dtc = DecisionTreeClassifier() # インスタンス作成
dtc.fit(X_train_transformed, y_train) # 学習
dtc_pred = dtc.predict(X_test_transformed) # 予測
dtc_pred

array([-1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,
       -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
       -1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
       -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1.,  1.])

### シンプルデータセット2での学習・推定

In [21]:
# サンプルコードでシンプルデータセット2作成

X = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
[ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
[-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
[ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
[-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
[-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
[-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
[-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
[ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
[-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
[-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
[ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
[ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
[-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
[ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
[-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
[ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
[-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
[ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
[-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
display(X.shape)
display(y.shape)

(40, 2)

(40,)

In [22]:
# スクラッチしたtrain_test_split関数でtrainデータとtestデータに分ける

X_train, X_test, y_train, y_test = scratch_train_test_split(X, y, train_size=0.75)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

X_train (30, 2)
X_test (10, 2)
y_train (30,)
y_test (10,)


In [23]:
# 前処理・標準化

scaler = StandardScaler() # インスタンス作成
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train) #X_trainの標準化
X_test_transformed = scaler.transform(X_test) #X_testの標準化
# print(X_train_transformed)
# print(X_test_transformed)

In [24]:
# シンプルデータセット1の学習・推定：ロジスティック回帰(勾配降下法SGDClassifierクラス使用)

sgdc = SGDClassifier(loss="log")
sgdc.fit(X_train_transformed, y_train)
sgdc_pred = sgdc.predict(X_test_transformed)
sgdc_pred

array([0., 1., 0., 1., 1., 0., 1., 1., 1., 1.])

In [25]:
# シンプルデータセット1の学習・推定：SVM

svm = SVC() # インスタンス作成
svm.fit(X_train_transformed, y_train) # 学習
svm_pred = svm.predict(X_test_transformed) # 予測
svm_pred

array([0., 1., 0., 0., 1., 0., 1., 1., 1., 0.])

In [26]:
# シンプルデータセット1の学習・推定：決定木

dtc = DecisionTreeClassifier() # インスタンス作成
dtc.fit(X_train_transformed, y_train) # 学習
dtc_pred = dtc.predict(X_test_transformed) # 予測
dtc_pred

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

## 【問題3】 回帰問題を解くコードの作成

線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [27]:
# データセットを読み込み、説明変数GrLivAreaとYearBuilt、目的変数SalePriceをそれぞれ抽出
df = pd.read_csv('train.csv')
df = df[['GrLivArea', 'YearBuilt', 'SalePrice']]
# df.info()

In [28]:
# スクラッチしたtrain_test_split関数でtrainデータとtestデータに分ける

X_train, X_test, y_train, y_test = scratch_train_test_split(df.loc[:, ['GrLivArea', 'YearBuilt']].values, df.iloc[:, 2].values, train_size=0.75)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1095, 2)
(365, 2)
(1095,)
(365,)


In [29]:
# 学習・推定：線形回帰(勾配降下法SGDRegressorクラス使用)
from sklearn.linear_model import SGDRegressor

sgdr = SGDRegressor()
sgdr.fit(X_train, y_train)
sgdr_pred = sgdr.predict(X_test)
sgdr_pred

array([2.40346564e+15, 1.03743515e+15, 1.03787834e+15, 1.61999311e+15,
       1.62276305e+15, 1.22398428e+15, 1.73085139e+15, 1.07647165e+15,
       1.32835061e+15, 9.05487600e+14, 1.74999827e+15, 7.34699163e+14,
       1.48929375e+15, 2.36631270e+15, 1.74816668e+15, 1.06678569e+15,
       1.48432699e+15, 1.57682425e+15, 1.21305357e+15, 1.22761461e+15,
       2.37302281e+15, 1.82179749e+15, 7.97089423e+14, 1.21690550e+15,
       1.02841055e+15, 1.43644237e+15, 1.46679353e+15, 1.10765035e+15,
       1.18689016e+15, 1.34960264e+15, 1.03085838e+15, 2.22493364e+15,
       1.20456040e+15, 1.79799368e+15, 8.39759691e+14, 2.09099174e+15,
       1.34693664e+15, 1.73901216e+15, 1.24606386e+15, 1.22309447e+15,
       8.45948661e+14, 1.64179227e+15, 1.91176554e+15, 1.74999827e+15,
       1.73720655e+15, 1.67633804e+15, 2.11876171e+15, 1.53923155e+15,
       1.61275156e+15, 3.26089452e+15, 1.74242433e+15, 1.27433585e+15,
       8.06501814e+14, 1.08160118e+15, 1.07272365e+15, 1.89254413e+15,
      